<a href="https://colab.research.google.com/github/anushkapatwa/CSCI-167/blob/main/Notebooks/Chap07/7_2_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 7.2: Backpropagation**

This notebook runs the backpropagation algorithm on a deep neural network as described in section 7.4 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

First let's define a neural network.  We'll just choose the weights and biases randomly for now

In [2]:
def relu(x):
    return np.maximum(0, x)

def forward(x, all_weights, all_biases):
    h = x
    # hidden layers
    for W, b in zip(all_weights[:-1], all_biases[:-1]):
        h = relu(W @ h + b)
    # output layer (linear)
    out = all_weights[-1] @ h + all_biases[-1]
    return out


In [3]:
# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

Now let's run our random network.  The weight matrices $\boldsymbol\Omega_{0\ldots K}$ are the entries of the list "all_weights" and the biases $\boldsymbol\beta_{0\ldots K}$ are the entries of the list "all_biases"

We know that we will need the preactivations $\mathbf{f}_{0\ldots K}$ and the activations $\mathbf{h}_{1\ldots K}$ for the forward pass of backpropagation, so we'll store and return these as well.


In [5]:
import numpy as np

def relu(x):
    return np.maximum(0, x)

def compute_network_output(net_input, all_weights, all_biases):
    # Retrieve number of layers (excluding output)
    K = len(all_weights) - 1

    # Pre-activations and activations
    all_f = [None] * (K+1)
    all_h = [None] * (K+1)

    # Input is activation of layer 0
    all_h[0] = net_input

    # Forward pass through hidden layers
    for layer in range(K):
        # pre-activation
        all_f[layer] = np.matmul(all_weights[layer], all_h[layer]) + all_biases[layer]
        # activation
        all_h[layer+1] = relu(all_f[layer])

    # Output layer (linear, no activation)
    all_f[K] = np.matmul(all_weights[-1], all_h[K]) + all_biases[-1]

    # Final network output
    net_output = all_f[K]

    return net_output, all_f, all_h


In [8]:
# Number of hidden layers
K = 5
D = 6
D_i = 1
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Input and output layers
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size=(D,1))
all_biases[-1]= np.random.normal(size=(D_o,1))

# Intermediate hidden layers
for layer in range(1,K):
    all_weights[layer] = np.random.normal(size=(D,D))
    all_biases[layer] = np.random.normal(size=(D,1))


Now let's define a loss function.  We'll just use the least squares loss function. We'll also write a function to compute dloss_doutput

In [9]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output -y);

In [11]:
# Input
net_input = np.ones((1,1)) * 1.2

# Forward pass through network
net_output, all_f, all_h = compute_network_output(net_input, all_weights, all_biases)

# Target
y = np.ones((1,1)) * 20.0   # since D_o = 1

# Compute loss
loss = least_squares_loss(net_output, y)
print("y = %3.3f  Loss = %3.3f"%(y, loss))


y = 20.000  Loss = 12608.351


/tmp/ipython-input-4185488072.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("y = %3.3f  Loss = %3.3f"%(y, loss))


Now let's compute the derivatives of the network.  We already computed the forward pass.  Let's compute the backward pass.

In [12]:
def backward_pass(all_weights, all_biases, all_f, all_h, y):
    # Store gradients
    all_dl_dweights = [None] * (K+1)
    all_dl_dbiases  = [None] * (K+1)
    all_dl_df       = [None] * (K+1)
    all_dl_dh       = [None] * (K+1)

    # Derivative wrt network output
    all_dl_df[K] = np.array(d_loss_d_output(all_f[K], y))

    # Backward loop
    for layer in range(K, -1, -1):
        # Eq. 7.22: bias gradient
        all_dl_dbiases[layer] = np.array(all_dl_df[layer])   # copy

        # Eq. 7.23: weight gradient
        all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].T)

        # Eq. 7.25 (first part): derivative wrt activations
        all_dl_dh[layer] = np.matmul(all_weights[layer].T, all_dl_df[layer])

        if layer > 0:
            # Eq. 7.25 (second part): backprop through ReLU
            relu_grad = indicator_function(all_f[layer-1])   # 1 if >0, else 0
            all_dl_df[layer-1] = all_dl_dh[layer] * relu_grad

    return all_dl_dweights, all_dl_dbiases


In [17]:
def indicator_function(x):
    x_in = np.array(x, copy=True)   # make a copy to avoid modifying original
    x_in[x_in > 0] = 1
    x_in[x_in <= 0] = 0
    return x_in
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)


In [18]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Test the dervatives of the bias vectors
for layer in range(K+1):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])
  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")



# Test the derivatives of the weights matrices
for layer in range(K+1):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[    0.   ]
 [ -650.398]
 [ 6087.484]
 [-1549.527]
 [   -0.   ]
 [ 2725.385]]
Bias 0, derivatives from finite differences
[[    0.   ]
 [ -650.398]
 [ 6087.485]
 [-1549.527]
 [    0.   ]
 [ 2725.386]]
Success!  Derivatives match.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[3667.49 ]
 [1070.949]
 [   0.   ]
 [  -0.   ]
 [ 677.495]
 [   0.   ]]
Bias 1, derivatives from finite differences
[[3667.49 ]
 [1070.949]
 [   0.   ]
 [   0.   ]
 [ 677.495]
 [   0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[ 173.404]
 [  33.02 ]
 [1079.331]
 [  -0.   ]
 [2225.069]
 [  -0.   ]]
Bias 2, derivatives from finite differences
[[ 173.404]
 [  33.02 ]
 [1079.331]
 [   0.   ]
 [2225.07 ]
 [   0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 3, derivatives from backprop: